# 简易NN实现性别预测

##### Copyright © 2020 by Wangchuwen，2018202114. All rights reserved.

## 一.构造神经网络

In [1]:
import numpy as np
import pandas as pd

def sigmoid(x):
  # Sigmoid activation function: f(x) = 1 / (1 + e^(-x))
  return 1 / (1 + np.exp(-x))

def deriv_sigmoid(x):
  # Derivative of sigmoid: f'(x) = f(x) * (1 - f(x))
  fx = sigmoid(x)
  return fx * (1 - fx)

def mse_loss(y_true, y_pred):
  # y_true and y_pred are numpy arrays of the same length.
  return ((y_true - y_pred) ** 2).mean()

class OurNeuralNetwork:
  '''
  A neural network with:
    - 2 inputs
    - a hidden layer with 2 neurons (h1, h2)
    - an output layer with 1 neuron (o1)

  *** DISCLAIMER ***:
  The code below is intended to be simple and educational, NOT optimal.
  Real neural net code looks nothing like this. DO NOT use this code.
  Instead, read/run it to understand how this specific network works.
  '''
  def __init__(self):
    # 权重，Weights
    self.w1 = np.random.normal()
    self.w2 = np.random.normal()
    self.w3 = np.random.normal()
    self.w4 = np.random.normal()
    self.w5 = np.random.normal()
    self.w6 = np.random.normal()

    # 截距项，Biases
    self.b1 = np.random.normal()
    self.b2 = np.random.normal()
    self.b3 = np.random.normal()


  def feedforward(self, x):
    # x is a numpy array with 2 elements.
    h1 = sigmoid(self.w1 * x[0] + self.w2 * x[1] + self.b1)
    h2 = sigmoid(self.w3 * x[0] + self.w4 * x[1] + self.b2)
    o1 = sigmoid(self.w5 * h1 + self.w6 * h2 + self.b3)
    if o1>0.5:
        return 1
    else:
        return 0

  def train(self, data, all_y_trues):
    '''
    - data is a (n x 2) numpy array, n = # of samples in the dataset.
    - all_y_trues is a numpy array with n elements.
      Elements in all_y_trues correspond to those in data.
    '''
    learn_rate = 0.2
    epochs = 10 # number of times to loop through the entire dataset

    for epoch in range(epochs):
      for x, y_true in zip(data, all_y_trues):
        # --- Do a feedforward (we'll need these values later)
        sum_h1 = self.w1 * x[0] + self.w2 * x[1] + self.b1
        h1 = sigmoid(sum_h1)

        sum_h2 = self.w3 * x[0] + self.w4 * x[1] + self.b2
        h2 = sigmoid(sum_h2)

        sum_o1 = self.w5 * h1 + self.w6 * h2 + self.b3
        o1 = sigmoid(sum_o1)
        y_pred = o1

        # --- Calculate partial derivatives.
        # --- Naming: d_L_d_w1 represents "partial L / partial w1"
        d_L_d_ypred = -2 * (y_true - y_pred)

        # Neuron o1
        d_ypred_d_w5 = h1 * deriv_sigmoid(sum_o1)
        d_ypred_d_w6 = h2 * deriv_sigmoid(sum_o1)
        d_ypred_d_b3 = deriv_sigmoid(sum_o1)

        d_ypred_d_h1 = self.w5 * deriv_sigmoid(sum_o1)
        d_ypred_d_h2 = self.w6 * deriv_sigmoid(sum_o1)

        # Neuron h1
        d_h1_d_w1 = x[0] * deriv_sigmoid(sum_h1)
        d_h1_d_w2 = x[1] * deriv_sigmoid(sum_h1)
        d_h1_d_b1 = deriv_sigmoid(sum_h1)

        # Neuron h2
        d_h2_d_w3 = x[0] * deriv_sigmoid(sum_h2)
        d_h2_d_w4 = x[1] * deriv_sigmoid(sum_h2)
        d_h2_d_b2 = deriv_sigmoid(sum_h2)

        # --- Update weights and biases
        # Neuron h1
        self.w1 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_w1
        self.w2 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_w2
        self.b1 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_b1

        # Neuron h2
        self.w3 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_w3
        self.w4 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_w4
        self.b2 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_b2

        # Neuron o1
        self.w5 -= learn_rate * d_L_d_ypred * d_ypred_d_w5
        self.w6 -= learn_rate * d_L_d_ypred * d_ypred_d_w6
        self.b3 -= learn_rate * d_L_d_ypred * d_ypred_d_b3

      # --- Calculate total loss at the end of each epoch
      if epoch % 1 == 0:
        y_preds = np.apply_along_axis(self.feedforward, 1, data)
        loss = mse_loss(all_y_trues, y_preds)
        print("Epoch %d loss: %.3f" % (epoch, loss))
        print(y_preds)

  def predict(self,X):
        y_pre = np.apply_along_axis(self.feedforward, 1, X)
        return y_pre   
        

    


## 二.装载数据集

In [2]:
url='classdata.csv'
dataframe=pd.read_csv(url)

## 三.数据预处理

In [3]:
def judge(x):
    if '男' == x:
        return 1
    elif '女'== x:
        return 0
dataframe['MW'] = dataframe.S.apply(lambda x: judge(x))

## 四.分割训练集和测试集

In [4]:
data=dataframe.values
X=data[:,1:3]
Y=data[:,4]
X_train=np.array(X)[1::2,]
X_test=np.array(X)[0::2,]
Y_train=np.array(Y)[1::2,]
Y_test=np.array(Y)[0::2,]
print(X_train)
print(X_test)
print(Y_train)
print(Y_test)

[[175 62.5]
 [180 65.0]
 [170 55.0]
 [172 60.0]
 [158 46.0]
 [167 57.0]
 [158 51.0]
 [160 51.0]
 [175 68.0]
 [163 57.5]
 [165 52.0]
 [178 63.0]
 [174 63.0]
 [182 69.0]
 [164 70.0]
 [162 54.0]
 [173 54.0]
 [175 84.0]
 [176 75.0]
 [174 72.0]
 [178 65.0]
 [177 60.0]]
[[163 54.0]
 [173 70.0]
 [159 43.0]
 [181 61.0]
 [174 61.0]
 [176 76.0]
 [184 74.0]
 [162 47.0]
 [158 48.0]
 [163 50.5]
 [170 68.0]
 [184 86.0]
 [161 55.0]
 [175 60.0]
 [175 50.0]
 [178 65.0]
 [173 65.0]
 [164 44.0]
 [173 58.0]
 [174 60.0]
 [180 66.0]
 [170 55.0]
 [160 55.0]]
[1 1 0 1 0 0 0 0 1 1 0 1 1 1 1 0 0 1 1 1 0 1]
[0 1 0 1 1 1 1 0 0 0 1 1 0 1 1 1 1 0 1 1 1 1 0]


## 五.训练神经网络

In [5]:
# Train our neural network!
network = OurNeuralNetwork()
network.train(X_train,Y_train)

Epoch 0 loss: 0.409
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Epoch 1 loss: 0.409
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Epoch 2 loss: 0.409
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Epoch 3 loss: 0.409
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Epoch 4 loss: 0.409
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Epoch 5 loss: 0.409
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Epoch 6 loss: 0.409
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Epoch 7 loss: 0.409
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Epoch 8 loss: 0.409
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Epoch 9 loss: 0.409
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


## 六.测试集上的分类效果

In [6]:
Y_pred=network.predict(X_test)
Y_pred
l = mse_loss(Y_test, Y_pred)
print("loss: %.3f" %l)


loss: 0.348
